In [1]:
import numpy as np

In [ ]:
def GetSEIR(currState,currVac,currM,infFor,L,Z,D,betaBase):#SEIR迭代方程（1天）
    newState=np.zeros((4,42))
    newcase=np.zeros(42)
    flag=0#越界=1
    delta_S=currState[0]*(currM.dot(infFor)[:,0])+betaBase*currState[0]
    test=currState[0]-delta_S-currVac[0]-currVac[1]+currState[3]/L
    if np.min(test)>=0:
        newState[0] = test
        newState[1] = currState[1]+delta_S-currState[1]/Z
        newState[2] =currState[2]+currState[1]/Z-currState[2]/D
        newcase=currState[1]/Z
        newState[3] = currState[3]+currVac[0]+currVac[1]+currState[2]/D -currState[3]/L
    else:
        flag=1
        print('已拦截越界并返回1')               
    return newState,newcase,flag


def GetLambda(I,N,v_set,a,b,currdensity,currdwell):#获得当前时刻，各个区各个类别的传播系数
    mylambda=np.zeros((42*cluster_num))
    repeat_v=np.tile(v_set,(1,42))#由于V是各区通用，复制42份即可
    partial=I/N
    mylambda=repeat_v*partial*(currdensity**a)*(currdwell**b)#0-6是第0区7个类别的传播系数，7-13是第1区7个类别的传播系数
    return mylambda.T

def simulation(StateStart,para):#根据传入的参数和初始状态，模拟传播
    """
    StateStart:初始状态
    para:参数
    模拟过程，返回每天每个区的四个状态的人数，以及每天每个区的感染案例
    """
    beta=para[0]
    a=0.5
    b=0.5
    v_set=para[1:]
    flag=0
    State=np.zeros((trans_tstop,4,42))#保存每天每个区的四个状态的人数
    State[0]=StateStart
    case=np.zeros((trans_tstop,42))  #保存每天每个区的模拟的感染案例
    uhf_daily_obs=np.zeros(((trans_tstop,42)))#每天每个区模拟确诊数
    uhf_week_obs=np.zeros(((13,42)))#按周保存每个区的模拟确诊数
    R0=0#第七天的再生数
    for t in range(1,trans_tstop):
        deltacase=np.zeros(42)
        I_set=State[t-1][2].dot(M_norm[t-1])
        N_set=(N_all.T).dot(M_norm[t-1])
        infFor=GetLambda(I_set,N_set,v_set,a,b,density[day2week[t-1]],dwell)#获得传播系数
        State[t],deltacase,flag=GetSEIR(State[t-1],vaccine[t-1],M_norm[t-1],infFor,L,Z,D,beta)#获得当前时刻的状态、新增感染、以及是否越界的判断
        if t ==7:
            R0=4*np.sum(deltacase)/np.sum(State[t-1][2])*np.sum(N_all)/np.sum(State[t-1][0])#获得R0、
        if flag==0:
            deltacase=deltacase.reshape(1,42)
            prob_case=prob_gamma.dot(deltacase)#延迟模型
            case[t:min(t+20,M_t)]+=prob_case[:min(M_t-t,20)]#按延迟情况统计感染案例
    uhf_daily_obs=sim_daily_rate*case #结合观测率，获得42区确诊案例
    sum_daily_obs=np.sum(uhf_daily_obs,axis=1)#获得NYC当天确诊案例
    uhf_week_obs=np.sum(uhf_daily_obs[4:-1].reshape((13,7,42)),axis=1)#按周统计确诊，计算42个区的似然性
    return flag,uhf_week_obs,sum_daily_obs,R0

def GetUHFLogLikelihood(real_uhf,uhf_obs):
    """
    real_uhf:真实确诊数
    uhf_obs:模拟确诊数
    返回不同参数下面的似然性
    """
    loguhflikelihood=0
    for i in range(42):
        #按确诊最大值的20%以上的几周来计算似然性，希望尽可能拟合峰值
        likelihood=scipy.stats.norm.pdf(real_uhf[date_scale[i][0]:date_scale[i][1]+1,i],loc=uhf_obs[date_scale[i][0]:date_scale[i][1]+1,i],scale=100+0.5*uhf_obs[date_scale[i][0]:date_scale[i][1]+1,i])
        likelihood[likelihood==0]=0.0000001
        loguhflikelihood+=np.sum(np.log(likelihood))#取对数并求和
        
    return loguhflikelihood

def MH_step(para_index,para_set,statestart,real_case,p_scale):
    """
    para_index:当前参数的索引
    para_set:当前参数集
    statestart:初始状态
    real_case:真实案例
    p_scale:参数阈值
    """
    prior_p=para_set[para_index]#先验参数

    pos_uhf_case=np.zeros((13,42))#用来保存后验参数产生的42区预测值
    pri_uhf_case=np.zeros((13,42))#保存先验参数产生的42区预测值

    pri_cell=simulation(statestart,para_set) ##先验参数下的模拟 
    pri_uhf_case=pri_cell[1]
    r0=pri_cell[3]#先验R0
    pri_log_prob=GetUHFLogLikelihood(real_case,pri_uhf_case)#获得先验似然性
    scale=p_scale[para_index]#获得当前参数阈值
    new_para_set=para_set.copy()#新的参数集
    repeat_num=0
    choice_flag=0
    
    while True:
        repeat_num+=1
        if repeat_num<100:
            while True:
                post_p=np.random.normal(loc=prior_p,scale=max(0.2*prior_p,0.00005))
                if post_p>=scale[0] and post_p<=scale[1]:#后验参数落在合理范围内
                    break
            new_para_set[para_index]=post_p
            iter_cell=simulation(statestart,new_para_set)
            if iter_cell[0]==0:#后验参数合理，没有造成越界情况
                pos_uhf_case=iter_cell[1]
                pos_log_prob=GetUHFLogLikelihood(real_case,pos_uhf_case)#后验参数的似然性
                r0=iter_cell[3]
                break 
        else:
            choice_flag=1#当前参数选了100次还没有合适的，说明当前位置有问题
            break

    if choice_flag==0:
        r=np.exp(pos_log_prob-pri_log_prob)#取指数
        if r>1 or np.random.uniform(0,1)<r:#按一定概率接受
            return post_p,1,r0
        else:
            return prior_p,0,r0
    else:
        print('Currently, there are no suitable posterior parameters, and the adaptation fails')
        return prior_p,2,r0#返回报错信息，程序暂停